In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

In [0]:
import gzip
import os
import pandas as pd
from os import listdir

path=os.getcwd()+'/drive//My Drive/Colab Notebooks/er_result/'


for i in listdir(path):
  df = pd.read_json(path+i, lines=True, compression='gzip')
  df

In [0]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql import types

In [0]:
spark = SparkSession.builder.appName('er').getOrCreate()
spark.sparkContext.setLogLevel('WARN')

path=os.getcwd()+'/drive//My Drive/Colab Notebooks/er_resume_preprocessed/'

job_details=spark.read.option('compression', 'gzip').json(path)
job_details.registerTempTable("job_details")

In [0]:
path=os.getcwd()+'/drive//My Drive/Colab Notebooks/er_titles_preprocessed/'

job_titles=spark.read.option('compression', 'gzip').json(path)
job_titles.registerTempTable("job_titles")

In [0]:
spark = SparkSession.builder.appName('er').getOrCreate()
spark.sparkContext.setLogLevel('WARN')

# er_schema = types.StructType([
#     types.StructField('Alternate Title', types.ArrayType(types.StringType()), True),
#     types.StructField('O*NET-SOC Code', types.StringType(), True),
#     types.StructField('Title', types.StringType(), True),
#     types.StructField('id1', types.IntegerType(), True),
#     types.StructField('id2', types.IntegerType(),True),
#     types.StructField('jaccard', types.FloatType(), True),
#     types.StructField('job_titles', types.ArrayType(types.StringType()),True),
#     types.StructField('rank', types.IntegerType(), True),
#     types.StructField('resume_id', types.StringType(), True),
#     ])

path=os.getcwd()+'/drive//My Drive/Colab Notebooks/er_result/'

resume_ids = spark.read.option('compression', 'gzip').json(path)
resume_ids.registerTempTable("resume_ids")

In [12]:
joined_data=spark.sql("SELECT a.`Alternate Title`as alternate_title,a.`O*NET-SOC Code` as onet_code, a.Title, b.id1, b.id2, b.jaccard, b.resume_id, c.job_details, c.job_title FROM job_titles a JOIN resume_ids b ON a.id = b.id1 JOIN job_details c ON b.id2 = c.id")
joined_data.show()
joined_data.registerTempTable("data")

+--------------------+----------+--------------------+-----+---+------------------+----------------+--------------------+--------------------+
|     alternate_title| onet_code|               Title|  id1|id2|           jaccard|       resume_id|         job_details|           job_title|
+--------------------+----------+--------------------+-----+---+------------------+----------------+--------------------+--------------------+
|[software, system...|15-1199.02|Computer Systems ...| 5310| 26|0.6666666666666666|388daff777202464|[cairo, egypt, se...|[software, engineer]|
|[software, system...|15-1133.00|Software Develope...| 4887| 26|0.6666666666666666|388daff777202464|[cairo, egypt, se...|[software, engineer]|
|          [engineer]|17-3023.03|Electrical Engine...| 7466| 26|               0.5|388daff777202464|[cairo, egypt, se...|[software, engineer]|
|          [engineer]|17-2161.00|   Nuclear Engineers| 6787| 26|               0.5|388daff777202464|[cairo, egypt, se...|[software, engineer]|